In [1]:
import numpy as np
import polars as pl
import polars_plugin_option_pricing as m
print(f"polars_plugin_option_pricing version: {m.__version__}")

polars_plugin_option_pricing version: 0.1.0


In [2]:
def _range(e):
    start = e["start"]
    end = e["end"]
    n_step = e["n_step"]
    return np.linspace(start, end, n_step)

In [3]:
%%time

spots = {"start": 98, "end": 102, "n_step": 3}
strikes = {"start": 100, "end": 100, "n_step": 1}
mats = {"start": 1, "end": 10, "n_step": 1}
vols = {"start": 0.1, "end": 0.5, "n_step": 1}
rates = {"start": 0.00, "end": 0.04, "n_step": 1}
divs = {"start": 0.00, "end": 0.07, "n_step": 1}

spots = {"start": 80, "end": 120, "n_step": 25}
strikes = {"start": 80, "end": 120, "n_step": 100}
mats = {"start": 1, "end": 10, "n_step": 10}
vols = {"start": 0.1, "end": 0.5, "n_step": 5}
rates = {"start": 0.00, "end": 0.04, "n_step": 5}
divs = {"start": 0.00, "end": 0.07, "n_step": 8}


df_is_call = pl.DataFrame(data=[True, False], schema=["is_call"])
df_spot = pl.DataFrame(data=_range(spots), schema={"spot": pl.Float32})
df_strike = pl.DataFrame(data=_range(strikes), schema={"strike": pl.Float32})
df_mat = pl.DataFrame(data=_range(mats), schema={"mat": pl.Float32})
df_vol = pl.DataFrame(data=_range(vols), schema={"vol": pl.Float32})
df_rate = pl.DataFrame(data=_range(rates), schema={"rate": pl.Float32})
df_div = pl.DataFrame(data=_range(divs), schema={"div": pl.Float32})


CPU times: user 744 μs, sys: 950 μs, total: 1.69 ms
Wall time: 1.45 ms


In [4]:
%%time

df = (
    df_is_call.join(df_spot, how="cross")
    .join(df_strike, how="cross")
    .join(df_mat, how="cross")
    .join(df_vol, how="cross")
    .join(df_rate, how="cross")
    .join(df_div, how="cross")
)
print(df)

shape: (10_000_000, 7)
┌─────────┬───────┬────────┬──────┬─────┬──────┬──────┐
│ is_call ┆ spot  ┆ strike ┆ mat  ┆ vol ┆ rate ┆ div  │
│ ---     ┆ ---   ┆ ---    ┆ ---  ┆ --- ┆ ---  ┆ ---  │
│ bool    ┆ f32   ┆ f32    ┆ f32  ┆ f32 ┆ f32  ┆ f32  │
╞═════════╪═══════╪════════╪══════╪═════╪══════╪══════╡
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.0  │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.01 │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.02 │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.03 │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.04 │
│ …       ┆ …     ┆ …      ┆ …    ┆ …   ┆ …    ┆ …    │
│ false   ┆ 120.0 ┆ 120.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.03 │
│ false   ┆ 120.0 ┆ 120.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.04 │
│ false   ┆ 120.0 ┆ 120.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.05 │
│ false   ┆ 120.0 ┆ 120.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.06 │
│ false   ┆ 120.0 ┆ 120.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.07 │
└─────────┴───────┴────────┴──────┴─────┴──────┴──────┘
CPU times: user 157 ms, s

In [5]:
%%time

df = df.with_columns(
    input=pl.struct(["is_call", "spot", "strike", "mat", "vol", "rate", "div"])
)
print(df)

shape: (10_000_000, 8)
┌─────────┬───────┬────────┬──────┬─────┬──────┬──────┬─────────────────────────────────┐
│ is_call ┆ spot  ┆ strike ┆ mat  ┆ vol ┆ rate ┆ div  ┆ input                           │
│ ---     ┆ ---   ┆ ---    ┆ ---  ┆ --- ┆ ---  ┆ ---  ┆ ---                             │
│ bool    ┆ f32   ┆ f32    ┆ f32  ┆ f32 ┆ f32  ┆ f32  ┆ struct[7]                       │
╞═════════╪═══════╪════════╪══════╪═════╪══════╪══════╪═════════════════════════════════╡
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.0  ┆ {true,80.0,80.0,1.0,0.1,0.0,0.… │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.01 ┆ {true,80.0,80.0,1.0,0.1,0.0,0.… │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.02 ┆ {true,80.0,80.0,1.0,0.1,0.0,0.… │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.03 ┆ {true,80.0,80.0,1.0,0.1,0.0,0.… │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.04 ┆ {true,80.0,80.0,1.0,0.1,0.0,0.… │
│ …       ┆ …     ┆ …      ┆ …    ┆ …   ┆ …    ┆ …    ┆ …                    

In [6]:
%%time

df = df.with_columns(output=m.black_scholes(
    "is_call", "spot", "strike", "mat", "vol", "rate", "div"
))
print(df)

shape: (10_000_000, 9)
┌─────────┬───────┬────────┬──────┬───┬──────┬──────┬───────────────────────┬──────────────────────┐
│ is_call ┆ spot  ┆ strike ┆ mat  ┆ … ┆ rate ┆ div  ┆ input                 ┆ output               │
│ ---     ┆ ---   ┆ ---    ┆ ---  ┆   ┆ ---  ┆ ---  ┆ ---                   ┆ ---                  │
│ bool    ┆ f32   ┆ f32    ┆ f32  ┆   ┆ f32  ┆ f32  ┆ struct[7]             ┆ struct[19]           │
╞═════════╪═══════╪════════╪══════╪═══╪══════╪══════╪═══════════════════════╪══════════════════════╡
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ … ┆ 0.0  ┆ 0.0  ┆ {true,80.0,80.0,1.0,0 ┆ {true,0.05,-0.05,0.5 │
│         ┆       ┆        ┆      ┆   ┆      ┆      ┆ .1,0.0,0.…            ┆ 19939,0.48…          │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ … ┆ 0.0  ┆ 0.01 ┆ {true,80.0,80.0,1.0,0 ┆ {true,-0.05,-0.15,0. │
│         ┆       ┆        ┆      ┆   ┆      ┆      ┆ .1,0.0,0.…            ┆ 480061,0.4…          │
│ true    ┆ 80.0  ┆ 80.0   ┆ 1.0  ┆ … ┆ 0.0  ┆ 0.02 ┆ {true,80.0,80.

In [7]:
%%time

df = df.drop(["is_call"]).unnest("output")
print(df)

shape: (10_000_000, 26)
┌───────┬────────┬──────┬─────┬───┬─────────────┬────────────┬────────┬───────────┐
│ spot  ┆ strike ┆ mat  ┆ vol ┆ … ┆ rho         ┆ voma       ┆ payoff ┆ pv_payoff │
│ ---   ┆ ---    ┆ ---  ┆ --- ┆   ┆ ---         ┆ ---        ┆ ---    ┆ ---       │
│ f32   ┆ f32    ┆ f32  ┆ f32 ┆   ┆ f32         ┆ f32        ┆ f32    ┆ f32       │
╞═══════╪════════╪══════╪═════╪═══╪═════════════╪════════════╪════════╪═══════════╡
│ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 38.404892   ┆ -0.796888  ┆ 0.0    ┆ 0.0       │
│ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 35.230583   ┆ 2.366876   ┆ 0.0    ┆ 0.0       │
│ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 32.103497   ┆ 11.600042  ┆ 0.0    ┆ 0.0       │
│ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 29.053547   ┆ 26.266823  ┆ 0.0    ┆ 0.0       │
│ 80.0  ┆ 80.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 26.108416   ┆ 45.426395  ┆ 0.0    ┆ 0.0       │
│ …     ┆ …      ┆ …    ┆ …   ┆ … ┆ …           ┆ …          ┆ …      ┆ …         │
│ 120.0 ┆ 120.0  ┆ 10.0 ┆ 0.5 ┆ … ┆ -616.549316 ┆ -9